# Práctica: Supresión de Feedback Acústico

**Objetivo:** Implementar una solución robusta para eliminar el feedback acústico (acople) que se produce en un intercomunicador full-duplex, especialmente en el escenario de cerrar la pantalla de un ordenador portátil.

**Solución Propuesta:** Desplazamiento de Frecuencia (Frequency Shifting) con una interfaz de control en tiempo real.

## 📖 Fundamento Teórico

### El Problema: El Bucle de Feedback Acústico

El feedback, también conocido como "acople" o efecto Larsen, es un fenómeno que ocurre cuando un micrófono capta el sonido emitido por un altavoz que está reproduciendo la propia señal de ese micrófono, creando un bucle cerrado.


El proceso es el siguiente:
1.  El **micrófono** capta un sonido.
2.  La señal viaja al **amplificador** (nuestro programa), que la procesa y la envía al altavoz.
3.  El **altavoz** reproduce el sonido.
4.  El sonido del altavoz viaja por el aire y es captado de nuevo por el **micrófono**, cerrando el bucle.

Si la ganancia del bucle es mayor que uno para una determinada frecuencia, esa frecuencia se amplificará exponencialmente en cada ciclo, resultando en el característico pitido agudo y creciente que queremos eliminar. Este problema se agrava drásticamente al cerrar la pantalla de un portátil, ya que la distancia entre micrófono y altavoces disminuye, aumentando la ganancia del bucle.

### Nuestra Solución: Desplazamiento de Frecuencia (Frequency Shifting)

Basándonos en la [teoría propuesta](https://tecnologias-multimedia.github.io/contents/feedback_suppression/), hemos elegido el método de **Desplazamiento de Frecuencia** por su robustez y eficacia, especialmente en entornos cambiantes.

La idea fundamental es "engañar" al bucle de feedback. En lugar de dejar que una frecuencia se amplifique a sí misma, modificamos ligeramente su tono (su frecuencia) justo antes de que se reproduzca.

El proceso es el siguiente:
1.  Recibimos el chunk de audio del interlocutor.
2.  Antes de enviarlo al altavoz, multiplicamos la señal de audio por una onda senoidal compleja (un fasor). Esta operación matemática tiene el efecto de desplazar todo el espectro de frecuencias de la señal unos pocos Hertzios hacia arriba o hacia abajo.
3.  El sonido que sale del altavoz ya está ligeramente alterado. Si el micrófono lo vuelve a captar, su frecuencia ya no es la misma que la que podría causar la resonancia.

Al cambiar constantemente la frecuencia en cada pasada por el bucle, es imposible que se forme una retroalimentación positiva sostenida. Es el equivalente a intentar empujar un columpio a un ritmo ligeramente incorrecto; nunca cogerá la altura máxima.

Hemos elegido un desplazamiento inicial de **7.0 Hz**, un valor lo suficientemente grande para romper el bucle pero tan pequeño que es prácticamente imperceptible para el oído humano, preservando así la calidad del audio. Sin embargo, debido a que cada hardware (micrófono, altavoces, tarjeta de sonido) responde de manera diferente, la cantidad óptima de desplazamiento puede variar. **Por esta razón, nuestra solución implementa un control deslizante (slider) que permite al usuario ajustar este valor en tiempo real, garantizando la máxima eficacia para cualquier equipo.**

In [1]:
# Importaciones y dependencias
# Asegúrate de tener las librerías instaladas:
# pip install numpy sounddevice pygame pygame-widgets

import numpy as np
import minimal
import buffer
import logging
import pygame
import pygame_widgets
from pygame_widgets.slider import Slider
from pygame_widgets.button import Button
import threading

# Configuramos los argumentos por defecto para que el notebook se pueda ejecutar sin parámetros de terminal
class Args:
    input_device = None
    output_device = None
    list_devices = False
    frames_per_second = 44100
    frames_per_chunk = 1024
    listening_port = 4444  # Puerto para la primera instancia
    destination_address = 'localhost'
    destination_port = 5555 # Puerto para la segunda instancia
    filename = None
    reading_time = None
    number_of_channels = 2
    show_stats = False
    show_samples = False
    show_spectrum = False
    buffering_time = 150

minimal.args = Args()

pygame 2.6.1 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Definición de la clase principal FeedbackSuppressor

class FeedbackSuppressor(buffer.Buffering__verbose):
    def __init__(self):
        super().__init__()
        
        pygame.init()

        self.suppression_active = True
        self.frequency_shift_hz = 7.0

        self.time_vector = np.arange(minimal.args.frames_per_chunk) / minimal.args.frames_per_second
        self.phasor = np.exp(2j * np.pi * self.frequency_shift_hz * self.time_vector)

        self.window_heigh = 200
        self.display = pygame.display.set_mode((512, self.window_heigh))
        pygame.display.set_caption("Control de Supresión de Feedback")

        self.slider = Slider(self.display, 50, 50, 400, 20, min=0, max=20, step=0.5, initial=self.frequency_shift_hz)
        self.button = Button(self.display, 200, 100, 100, 50, text='On', onClick=self.toggle_suppression)
        self.update_button_style()

        logging.info("Supresor de Feedback con control interactivo inicializado.")

    def toggle_suppression(self):
        self.suppression_active = not self.suppression_active
        logging.info(f"Supresión de Feedback: {'ACTIVADA' if self.suppression_active else 'DESACTIVADA'}")
        self.update_button_style()

    def update_button_style(self):
        if self.suppression_active:
            self.button.string = 'On'
            self.button.inactiveColour = (20, 180, 20)
            self.button.hoverColour = (50, 220, 50)
        else:
            self.button.string = 'Off'
            self.button.inactiveColour = (180, 20, 20)
            self.button.hoverColour = (220, 50, 50)

    def update_phasor(self):
        current_shift = self.slider.getValue()
        if current_shift != self.frequency_shift_hz:
            self.frequency_shift_hz = current_shift
            self.phasor = np.exp(2j * np.pi * self.frequency_shift_hz * self.time_vector)

    def shift_frequency(self, chunk):
        self.update_phasor()
        
        shifted_chunk = np.zeros_like(chunk, dtype=np.float64)
        for i in range(chunk.shape[1]):
            modulated_signal = chunk[:, i] * self.phasor
            shifted_chunk[:, i] = modulated_signal.real
        
        return shifted_chunk.astype(np.int16)

    def _record_IO_and_play(self, ADC, DAC, frames, time, status):
        self.chunk_number = (self.chunk_number + 1) % self.CHUNK_NUMBERS
        packed_chunk = self.pack(self.chunk_number, ADC)
        self.send(packed_chunk)
        
        chunk_to_play = self.unbuffer_next_chunk()

        try:
            chunk_to_play_2d = chunk_to_play.reshape(minimal.args.frames_per_chunk, minimal.args.number_of_channels)
        except ValueError:
            chunk_to_play_2d = self.zero_chunk

        if self.suppression_active:
            processed_chunk = self.shift_frequency(chunk_to_play_2d)
        else:
            processed_chunk = chunk_to_play_2d

        self.play_chunk(DAC, processed_chunk)
    
    def loop_update_display(self):
        font = pygame.font.SysFont(None, 24)
        while not self.end:
            events = pygame.event.get()
            for event in events:
                if event.type == pygame.QUIT:
                    self.end = True
                    break
            if self.end: break

            self.display.fill((20, 20, 20))
            
            slider_label = font.render(f'Desplazamiento: {self.slider.getValue():.1f} Hz', True, (255, 255, 255))
            self.display.blit(slider_label, (50, 25))

            pygame_widgets.update(events)
            pygame.display.update()
            pygame.time.wait(30)

    def run(self):
        cycle_feedback_thread = threading.Thread(target=self.loop_cycle_feedback)
        cycle_feedback_thread.daemon = True
        
        network_thread = threading.Thread(target=self.loop_receive_and_buffer)
        network_thread.daemon = True

        self.print_running_info()
        self.print_header()
        self.end = False
        
        self.played_chunk_number = 0

        with self.stream(self._handler):
            cycle_feedback_thread.start()
            network_thread.start()
            self.loop_update_display()

## 🧪 Cómo Comprobar el Funcionamiento

Para verificar que la supresión de feedback funciona correctamente, es necesario ejecutar dos instancias de este notebook simultáneamente.

**Instrucciones:**
1.  Abre este notebook y ejecútalo ("Run All"). Esta será la **Terminal 1**.
2.  Crea una copia de este notebook (ej. `feedback_suppression_copia.ipynb`).
3.  Abre la copia y modifica la celda de "Importaciones y Preparación" cambiando los puertos:
    * `minimal.args.listening_port = 5555`
    * `minimal.args.destination_port = 4444`
4.  Ejecuta este segundo notebook. Esta será la **Terminal 2**.

Una vez ambas instancias estén corriendo, aparecerán dos ventanas de control. Ahora puedes realizar la prueba:

> Para comprobar el funcionamiento del feedback supression recomiendo **cerrar la pantalla del ordenador** a la altura deseada. Después, **ajustar los hercios** de la terminal 1 y de la terminal 2 mediante el slider hasta encontrar el **punto dulce**, es decir, el punto donde se está cancelando el feedback. Aquí la prueba ya estaría finalizada, pero también tienes la opción de **desactivar el botón "On" y notar el feedback original** para luego encender nuevamente el botón y notar como directamente se silencia el feedback.

**Nota Importante:** Para detener la ejecución de cada intercomunicador, es necesario seleccionar su respectiva pestaña en el navegador y hacer clic en el botón de **"Interrumpir el kernel" (⏹️)** en la barra de herramientas de Jupyter.

In [ ]:
# Celda 3: Ejecución del intercomunicador
# Al ejecutar esta celda, se lanzará la aplicación.
# Para detenerla, deberás interrumpir el kernel del notebook (botón de Stop).

try:
    intercom = FeedbackSuppressor()
    intercom.run()
except KeyboardInterrupt:
    print("\nPrograma detenido.")
finally:
    pygame.quit()

(INFO) minimal: A minimal InterCom (no compression, no quantization, no transform, ... only provides a bidirectional (full-duplex) transmission of raw (playable) chunks. 
(INFO) minimal: chunk_time = 0.023219954648526078 seconds
(INFO) minimal: seconds_per_cycle = 1
(INFO) minimal: chunks_per_cycle = 43.06640625
(INFO) minimal: frames_per_cycle = 44100
(INFO) buffer: Over minimal, implements a random access buffer structure for hiding the jitter.
(INFO) buffer: buffering_time = 150 miliseconds
(INFO) buffer: chunks_to_buffer = 7
(INFO) 2178787303: Supresor de Feedback con control interactivo inicializado.



InterCom parameters:


Using device:

  0 Micrófono de “iPhone (52)”, Core Audio (1 in, 0 out)
> 1 Micrófono del MacBook Air, Core Audio (1 in, 0 out)
< 2 Altavoces del MacBook Air, Core Audio (0 in, 2 out)

Use CTRL+C to quit
         sent   recv.    sent    recv.   Global
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
    1      41       0     357       0    7    0
Avgs:      41       0     357       0    7    0
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

    2      44       0    1434       0    6   33
Avgs:      42       0     895       0    6   16
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

    3      43       0    1402       0    6   18
Avgs:      42       0    1064       0    6   17
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

    4      43       0    1407       0    8   15
Avgs:      42       0    1150       0    7   16
c

(INFO) 2178787303: Supresión de Feedback: DESACTIVADA


   22      43      43    1402    1402   11   31
Avgs:      43      11    1360     377    7   25
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   23      43      43    1402    1402   11   32
Avgs:      43      12    1362     421    7   26
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   24      43      43    1402    1402   10   26
Avgs:      43      14    1363     462    8   26
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   25      42      43    1369    1402   10   23
Avgs:      43      15    1364     500    8   26
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   26      44      43    1438    1405   10   27
Avgs:      43      16    1366     534    8   26
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

-   27      44      43    1438    1

(INFO) 2178787303: Supresión de Feedback: ACTIVADA


...

   41      43      43    1402    1402   11   33
Avgs:      43      26    1382     854    9   27
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   42      43      44    1402    1435   11   29
Avgs:      43      26    1383     868    9   27
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   43      44      43    1435    1402   11   30
Avgs:      43      26    1384     881    9   27
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   44      43      43    1403    1403   10   28
Avgs:      43      27    1384     893    9   27
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   45      42      42    1369    1369   11   24
Avgs:      43      27    1384     903    9   27
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   46      43      43    1406    14

(INFO) 2178787303: Supresión de Feedback: ACTIVADA


   48      43      43    1402    1402   12   28
Avgs:      43      28    1385     934    9   27
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   49      44      44    1435    1435   12   29
Avgs:      43      28    1386     945    9   27
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   50      43      43    1402    1402   10   33
Avgs:      43      29    1386     954    9   27
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   51      42      42    1369    1369   10   24
Avgs:      43      29    1386     962    9   27
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   52      43      44    1407    1440   11   24
Avgs:      43      29    1386     971    9   27
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   53      43      42    1401    13

   96       8      41     260    1337    7   37
Avgs:      42      35    1383    1169   10   28
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   97       0      43       0    1403    3   30
Avgs:      42      35    1369    1172   10   28
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   98       0      44       0    1435    3   38
Avgs:      41      35    1355    1174   10   28
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

   99       0      43       0    1405    3   29
Avgs:      41      36    1342    1177    9   28
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

  100       0      43       0    1402    3   34
Avgs:      40      36    1328    1179    9   28
cycle  mesgs.  mesgs.    KBPS    KBPS %CPU %CPU
         sent   recv.    sent    recv.   Global

  101       0      43       0    14

: 

# Participantes del Proyecto

---

## 👥 Participantes

* **Ivelin Iliyanov Apostolov**
* **Samuel Mancebo Ortega**
* **Daniel García Gualda**
* **Lorenzo Valentín Cretu Abutnaritei**